In [ ]:
#| default_exp model

# EVCI siting model

> **API**: The mathematical formulation is defined here.

In [ ]:
#|hide
from nbdev.showdoc import *

In [ ]:
#|export

import numpy as np
import pandas as pd
import geopandas as gpd

import shapely

import os
from tqdm import tqdm

from evci_tool.config import *

import warnings
warnings.filterwarnings("ignore")

In [ ]:
#|export
def score(r,s_df_distances,j,i,hj,k,backoff=True, backoff_factor=1):
    "This function computes the utilization score of each site."
    
    distance_from_i = s_df_distances[s_df_distances > 0][i].sort_values()/1e3
    closer_to_i = distance_from_i[distance_from_i <= 5.0]
    try:
        congestion = float(s_df_distances.loc[i]['Traffic congestion'])
    except:
        congestion = 1.0

    nw = r['qjworking'][j][hj] * r['djworking'][j][hj] * r['pj'][k] * congestion
    nh = r['qjholiday'][j][hj] * r['djholiday'][j][hj] * r['pj'][k] * congestion

    if backoff:
        for el in closer_to_i:
            nw *= (1 - np.exp(-el*backoff_factor))
            nh *= (1 - np.exp(-el*backoff_factor))

    tw = th = 0
    if (r['Cij'][j][i] > 0): tw = nw * (r['tj'][j]/r['Cij'][j][i])
    if (r['Cij'][j][i] > 0): th = nh * (r['tj'][j]/r['Cij'][j][i])

    uw = uh = r['tj'][j]
    if (tw <= r['tj'][j]): uw = tw 
    if (th <= r['tj'][j]): uh = th

    vw = vh = 0
    if (tw > r['tj'][j]): vw = (tw - r['tj'][j]) * (r['Cij'][j][i]/r['tj'][j])
    if (th > r['tj'][j]): vh = (th - r['tj'][j]) * (r['Cij'][j][i]/r['tj'][j])

    norm_uw, norm_uh = uw/r['tj'][j], uh/r['tj'][j]
    
    if nw>0: norm_vw = vw/nw
    else: norm_vw = 0
    if nh>0: norm_vh = vh/nh
    else: norm_vh = 0

    return norm_uw, norm_uh, norm_vw, norm_vh

`Arguments`:

1. `r`: a dictionary of global parameters read from the xlsx files.
2. `s_df_distances`: a dataframe of Euclidean distances of each site from all others. (NxN matrix)
3. `j`: string indicating specific charger type
4. `i`: integer indicating a specific site
5. `hj`: 
6. `k`: integer year (1, 2 or 3 of the policy)
7. `backoff`: a boolean indicating whether backoff should be incorporated
8. `backoff_factor`: a float weighting factor for the backoff (mostly empirically selected)

`Returns`:

1. `norm_uw`: float indicating normalized utilization on a typical working day
2. `norm_uh`: float indicating normalized utilization on a typical holiday
3. `norm_vw`: float indicating number of vehicles not utilizing charging on a working day
4. `norm_vh`: float indicating number of vehicles not utilizing charging on a holiday

In [ ]:
#|export

def capex(r,i):
    "This function computes the capex requirements of each site"
    retval = 0
    for j in r['C']:
        retval += r['Cij'][j][i]*r['Kj'][j] + r['Wi'][i] * r['di'][i] * r['Cij'][j][i]
    return retval

`Arguments`:

1. `r`: a dictionary of global parameters read from the xlsx files.
2. `i`: integer indicating a specific site

`Returns`:

integer. Capex value for a given site

In [ ]:
#|export

def opex(r,s_df_distances,i):
    "This function computes the opex for each site."
    op_e = 0
    op_l = 0

    for k in r['years_of_analysis']:
        for j in r['C']:
            for h in range(int(r['timeslots'][j])):
                sw, sh, _, _ = score (r,s_df_distances,j,i,h,k)
                op_e += 300 * r['Cij'][j][i] * sw * r['tj'][j] * r['Dj'][j] * (r['l'][j][h] * r['Eg'][j][h] + (1-r['l'][j][h]) * r['Er'][j][h])
                op_e +=  65 * r['Cij'][j][i] * sh * r['tj'][j] * r['Dj'][j] * (r['l'][j][h] * r['Eg'][j][h] + (1-r['l'][j][h]) * r['Er'][j][h])
    op_l = r['Li'][i] * r['Ai'][i] + r['CH'][i] + r['CK'][i]
    return op_e + op_l

`Arguments`:

1. `r`: a dictionary of global parameters read from the xlsx files.
2. `s_df_distances`: a dataframe of Euclidean distances of each site from all others. (NxN matrix)
3. `i`: integer indicating a specific site

`Returns`:

integer opex value for a given site

In [ ]:
#|export

def margin(r,s_df_distances,i):
    "This function computes the margins per site."
    margin_e = 0
    margin_l = 0

    for k in r['years_of_analysis']:
        for j in r['C']:
            for h in range(int(r['timeslots'][j])):
                sw, sh, _, _ = score (r,s_df_distances,j,i,h,k)
                margin_e += 300 * r['Cij'][j][i] * sw * r['tj'][j] * r['Dj'][j] * (r['l'][j][h] * r['Mg'][j][h] + (1-r['l'][j][h]) * r['Mr'][j][h])
                margin_e +=  65 * r['Cij'][j][i] * sh * r['tj'][j] * r['Dj'][j] * (r['l'][j][h] * r['Mg'][j][h] + (1-r['l'][j][h]) * r['Mr'][j][h])
    margin_l = r['Bi'][i] * r['Ai'][i] + r['MH'][i] + r['MK'][i]
    return margin_e + margin_l

`Arguments`:

1. `r`: a dictionary of global parameters read from the xlsx files.
2. `s_df_distances`: a dataframe of Euclidean distances of each site from all others. (NxN matrix)
3. `i`: integer indicating a specific site

`Returns`:

integer margin value of a site

In [ ]:
#|export

def run_analysis(m,s,t,g,ui_inputs,s_df,backoff_factor=1):
    "This function runs analysis for a given set of sites."

    r = read_globals(m,s,t,g,ui_inputs)
    
    u_df = pd.DataFrame(columns=['utilization', 
                             'unserviced', 
                             'capex', 
                             'opex', 
                             'margin', 
                             'max vehicles', 
                             'estimated vehicles'
                             ])    
    s_df_crs = gpd.GeoDataFrame(s_df, crs='EPSG:4326')
    s_df_crs = s_df_crs.to_crs('EPSG:5234')
    s_df_distances = s_df_crs.geometry.apply(lambda g: s_df_crs.distance(g))      
    
    s_df_distances['Traffic congestion'] = s['sites']['Traffic congestion']
    
    Nc = s_df.shape[0]
    
    for i in tqdm(range(Nc)):
        max_vehicles = 0
        # run through selected charger types
        for j in r['M']:
            max_vehicles += r['timeslots'][j]*r['Cij'][j][i]
        max_vehicles = int(np.round(max_vehicles,0))
        op_e = 0
        op_l = 0
        margin_e = 0
        margin_l = 0
        year_u_avg = np.array([])
        year_v_avg = np.array([])
        for k in r['years_of_analysis']:
            chargertype_u_avg = np.array([])
            chargertype_v_avg = np.array([])
            for j in r['C']:
                uw_day_avg = np.array([])
                uh_day_avg = np.array([])
                vw_day_avg = np.array([])
                vh_day_avg = np.array([])              
                for h in range(int(r['timeslots'][j])):
                    uw, uh, vw, vh = score (r,s_df_distances,j,i,h,k,backoff_factor=backoff_factor)
                    uw_day_avg = np.append(uw_day_avg, uw)
                    uh_day_avg = np.append(uh_day_avg, uh)
                    vw_day_avg = np.append(vw_day_avg, vw)
                    vh_day_avg = np.append(vh_day_avg, vh)                  
                    op_e += 300 * r['Cij'][j][i] * uw * r['tj'][j] * r['Dj'][j] * (r['l'][j][h] * r['Eg'][j][h] + (1-r['l'][j][h]) * r['Er'][j][h])
                    op_e +=  65 * r['Cij'][j][i] * uh * r['tj'][j] * r['Dj'][j] * (r['l'][j][h] * r['Eg'][j][h] + (1-r['l'][j][h]) * r['Er'][j][h])            
                    margin_e += 300 * r['Cij'][j][i] * uw * r['tj'][j] * r['Dj'][j] * (r['l'][j][h] * r['Mg'][j][h] + (1-r['l'][j][h]) * r['Mr'][j][h])
                    margin_e +=  65 * r['Cij'][j][i] * uh * r['tj'][j] * r['Dj'][j] * (r['l'][j][h] * r['Mg'][j][h] + (1-r['l'][j][h]) * r['Mr'][j][h])        
                weighted_u = (300.0*uw_day_avg.mean() + 65.0*uh_day_avg.mean()) / 365.0
                weighted_v = (300.0*vw_day_avg.mean() + 65.0*vh_day_avg.mean()) / 365.0
                chargertype_u_avg = np.append(chargertype_u_avg, weighted_u)
                chargertype_v_avg = np.append(chargertype_v_avg, weighted_v)
            year_u_avg = np.append(year_u_avg, chargertype_u_avg.mean())
            year_v_avg = np.append(year_v_avg, chargertype_v_avg.mean())
            op_l += r['Li'][i] * r['Ai'][i] + r['CH'][i] + r['CK'][i]
            margin_l += r['Bi'][i] * r['Ai'][i] + r['MH'][i] + r['MK'][i]
        site_capex = capex(r,i)
        estimated_vehicles = np.round(year_u_avg.mean()*max_vehicles,0)
        u_df.loc[i] = [ year_u_avg.mean(), 
                       year_v_avg.mean(),
                       site_capex,
                       op_e + op_l,
                       margin_e + margin_l,
                       max_vehicles,
                       estimated_vehicles
                       ]
    return u_df

`Arguments`:

1. `m`: dataframe of model parameters (from model.xlsx)
2. `s`: dataframe of sites (from sites.xlsx)
3. `t`: dataframe of traffic profile (from traffic.xlsx)
4. `g`: dataframe of grid parameters (from grid.xlsx)
5. `ui_inputs`: json object of user selected inputs from the UI
6. `s_df`: pre-processed geopandas dataframe with each point stored as shapely point object
7. `backoff_factor`: a float value of backoff to cater for neighborhood (empirical)

`Returns`:

A dataframe of utilization values for all sites.